In [1]:
import pandas as pd

In [2]:
df_normal  = pd.read_csv('data.csv') # Load các thông tin các video về âm nhạc thường
df_normal.head()

,id,title,description,channelTitle,categoryId,tags,defaultAudioLanguage,publishedAt,duration,definition,caption,viewCount,likeCount,commentCount,favoriteCount,topicCategories,madeForKids,privacyStatus,embeddable,defaultLanguage
0,0nljmuN19DU,Nhạc Tập GYM Cực Phê Xem Chỉ Muốn Đi Tập Ngay,Follow kênh Tiktok của Ad nhé: https://www.tik...,Tôi Yêu GYM,17,"[""thể hình"",""idol gym"",""jeffseid"",""Ryan Terry""]",vi,2018-09-18T02:23:17Z,PT29M29S,hd,False,6258052.0,23333.0,751.0,0,"[""https://en.wikipedia.org/wiki/Lifestyle_(soc...",False,public,True,NaN
1,dxrYz563OdY,NHẠC TẬP GYM💪NHẠC TẬP GYM CỰC HAY 2020💪MUSIQUE...,►NHẠC TẬP GYM💪NHẠC TẬP GYM CỰC HAY 2020💪MUSIQU...,Đình Khởi Production,10,"[""nhạc tập gym"",""nhac tap gym"",""nhạc tập gym c...",NaN,2020-07-14T12:30:45Z,PT1H14M6S,hd,False,2685742.0,15861.0,390.0,0,"[""https://en.wikipedia.org/wiki/Electronic_mus...",False,public,True,NaN
2,GZj8vOfutTA,NHẠC TẬP GYM CỰC CHẤT 2023 | Best Workout Musi...,NHẠC TẬP GYM CỰC CHẤT 2023 | Best Workout Musi...,Nhạc Tập Gym,17,"[""nhạc tập gym"",""nhac tap gym"",""nhạc tập gym 2...",vi,2023-01-07T10:29:13Z,PT2H6M6S,hd,False,56056.0,220.0,1.0,0,"[""https://en.wikipedia.org/wiki/Electronic_mus...",False,public,True,vi
3,N6rS6lYZwTM,NHẠC EDM MIX TẬP GYM |#1| NHẠC TẬP GYM 2020 | ...,Đến Với Việt Hoàng Fitness Để Tìm Cảm Hứng Tập...,Việt Hoàng Fitness,22,NaN,NaN,2020-05-05T13:03:30Z,PT39M16S,hd,False,3147955.0,15207.0,475.0,0,"[""https://en.wikipedia.org/wiki/Health"",""https...",False,public,True,NaN
4,QhV1hEFKs3Q,Nhạc Tập Gym | Nhạc Tập Gym Cực Chất 2021 EDM ...,#nhactapgym #nhạctậpgym #nhactaptheduc #nhạctậ...,Gymer Việt Nam,22,NaN,NaN,2021-09-06T07:55:20Z,PT1H28M31S,sd,False,94097.0,336.0,4.0,0,"[""https://en.wikipedia.org/wiki/Electronic_mus...",False,public,True,NaN
